In [1]:
!pip -q install torch_geometric rectools
!pip -q install comet_ml
!pip -q install python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 46.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.0/208.0 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 107.3 MB/s eta 0:00:0000:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 1.42.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 727.1/727.1 kB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 48.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 88.7 MB/s eta 0:00:00:00:01


In [2]:
import comet_ml
from comet_ml import Experiment
from comet_ml.integration.pytorch import log_model

from dotenv import load_dotenv
import os

In [ ]:
load_dotenv(".env")

True

In [4]:
experiment = Experiment(
  api_key=os.getenv('API_KEY'),
  project_name="gnn-recommender",
  workspace="annanet",
  log_code=True
)

experiment.set_name('emogat-books')
experiment.add_tags(['books', 'leave-n-out'])

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/annanet/gnn-recommender/bed3511a1e964e0594d0fb04c7d195b3

COMET INFO: Couldn't find a Git repository in '/kaggle/working' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.


https://www.comet.com/annanet/gnn-recommender/bed3511a1e964e0594d0fb04c7d195b3

In [5]:
hyperparameters = {
    'seed': 42,
    'types_of_feedback': ["explicit_positive", "expliсit_negative",
                          "implicit_positive", "implicit_negative"],
    'train_edge_type': ('item','to_feedback_explicit_positive','explicit_positive'),
    'train_num_epochs': 100,
    'train_lr': 3e-4,
    'train_batch_size': 32768,
    'train_print_every': 25,  
    'train_test_every': 25,
    'test_topk': 10,
    'test_batch_size': 32768
}

In [6]:
import os
os.listdir('/kaggle/input/data/leave-n-out/books')

['train.csv', 'test.csv']

In [7]:
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import HeteroData
from torch_geometric.nn import HeteroConv, SAGEConv, GATConv

from sklearn.preprocessing import LabelEncoder

from rectools import Columns
from rectools.metrics import MAP, Precision, Recall, NDCG, calc_metrics

import gc
import random

In [8]:
SEED = hyperparameters['seed']
torch.manual_seed(SEED)
random.seed(SEED)
np.random.seed(SEED)

In [9]:
rootpath = '/kaggle/input/data/leave-n-out/books/'
train = pd.read_csv(
    rootpath+'train.csv'
)
train['date'] = pd.to_datetime(train['date_added'])
print(train.head())

                            user_id  book_id  rating  \
0  000883382802f2d95a3dd545bb953882  8525590       3   
1  000883382802f2d95a3dd545bb953882  2767052       5   
2  000883382802f2d95a3dd545bb953882  3236307       5   
3  000883382802f2d95a3dd545bb953882   256683       4   
4  000883382802f2d95a3dd545bb953882  6001758       5   

                  date_added                       date  
0  2011-10-31 18:37:56-07:00  2011-10-31 18:37:56-07:00  
1  2011-10-31 18:40:33-07:00  2011-10-31 18:40:33-07:00  
2  2011-11-02 08:30:30-07:00  2011-11-02 08:30:30-07:00  
3  2011-11-02 08:55:16-07:00  2011-11-02 08:55:16-07:00  
4  2011-11-02 08:57:21-07:00  2011-11-02 08:57:21-07:00  


/tmp/ipykernel_35/4238775169.py:5: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  train['date'] = pd.to_datetime(train['date_added'])


In [10]:
explicit_positive = train[(train["rating"] == 5)].index
explisit_negative = train[(train["rating"] <= 2)].index

explicit_combined_feedback = explicit_positive.union(explisit_negative)
print('Количество explicit позитивного фидбека', explicit_positive.shape[0])
print('Количество explicit негативного фидбека', explisit_negative.shape[0])

Количество explicit позитивного фидбека 359463
Количество explicit негативного фидбека 178217


In [11]:
implicit_positive = train[(train["rating"] == 4)].index
implicit_negative = train[(train["rating"] == 3)].index

implicit_combined_feedback = implicit_positive.union(implicit_negative)
print('Количество implicit позитивного фидбека', implicit_positive.shape[0])
print('Количество implicit негативного фидбека', implicit_negative.shape[0])

Количество implicit позитивного фидбека 430342
Количество implicit негативного фидбека 258651


In [12]:
train.loc[:, "target"] = ""
train.loc[explicit_positive, "target"] = "explicit_positive"
train.loc[explisit_negative, "target"] = "expliсit_negative"
train.loc[implicit_positive, "target"] = "implicit_positive"
train.loc[implicit_negative, "target"] = "implicit_negative"

train = train[['user_id','book_id','target','date']]
train.head()

,user_id,book_id,target,date
0,000883382802f2d95a3dd545bb953882,8525590,implicit_negative,2011-10-31 18:37:56-07:00
1,000883382802f2d95a3dd545bb953882,2767052,explicit_positive,2011-10-31 18:40:33-07:00
2,000883382802f2d95a3dd545bb953882,3236307,explicit_positive,2011-11-02 08:30:30-07:00
3,000883382802f2d95a3dd545bb953882,256683,implicit_positive,2011-11-02 08:55:16-07:00
4,000883382802f2d95a3dd545bb953882,6001758,explicit_positive,2011-11-02 08:57:21-07:00


In [13]:
train = train.sort_values(by=["user_id", "date"]).reset_index(drop=True)

In [14]:
test = pd.read_csv(
    rootpath+'test.csv'
)
test['date'] = pd.to_datetime(test['date_added'])
print(test.head())

                            user_id   book_id  rating  \
0  000883382802f2d95a3dd545bb953882   8135807       4   
1  000883382802f2d95a3dd545bb953882  18301124       5   
2  000883382802f2d95a3dd545bb953882  18220354       4   
3  000883382802f2d95a3dd545bb953882  17383918       5   
4  000883382802f2d95a3dd545bb953882  13188676       5   

                  date_added                       date  
0  2013-08-13 09:37:39-07:00  2013-08-13 09:37:39-07:00  
1  2013-10-27 22:18:01-07:00  2013-10-27 22:18:01-07:00  
2  2013-12-09 22:20:59-08:00  2013-12-09 22:20:59-08:00  
3  2013-12-22 20:57:14-08:00  2013-12-22 20:57:14-08:00  
4  2013-12-22 20:58:15-08:00  2013-12-22 20:58:15-08:00  


/tmp/ipykernel_35/2717431719.py:4: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  test['date'] = pd.to_datetime(test['date_added'])


In [15]:
test = test[['user_id','book_id', 'date']]
test.head()

,user_id,book_id,date
0,000883382802f2d95a3dd545bb953882,8135807,2013-08-13 09:37:39-07:00
1,000883382802f2d95a3dd545bb953882,18301124,2013-10-27 22:18:01-07:00
2,000883382802f2d95a3dd545bb953882,18220354,2013-12-09 22:20:59-08:00
3,000883382802f2d95a3dd545bb953882,17383918,2013-12-22 20:57:14-08:00
4,000883382802f2d95a3dd545bb953882,13188676,2013-12-22 20:58:15-08:00


In [16]:
train.columns = ['user_id', 'item_id', 'target', 'date']
test.columns = ['user_id', 'item_id', 'date']

# MVP model v2

In [17]:
test = test[(test.user_id.isin(train.user_id)) & (test.item_id.isin(train.item_id))].copy()
test.shape

(151126, 3)

In [18]:
# 2. Преобразование данных - для куарека не особо нужно, но для других - напоминалка
# делаем всегда! чтобы не сломать ничего дальше и чтобы все индексы были от 0 до N без пропусков
user_encoder = LabelEncoder()
video_encoder = LabelEncoder()

train.loc[:, 'user_id'] = user_encoder.fit_transform(train['user_id'])
train.loc[:, 'item_id'] = video_encoder.fit_transform(train['item_id'])

test.loc[:, 'user_id'] = user_encoder.transform(test['user_id'])
test.loc[:, 'item_id'] = video_encoder.transform(test['item_id'])

train['user_id'] = train['user_id'].astype(int)
train['item_id'] = train['item_id'].astype(int)
test['user_id'] = test['user_id'].astype(int)
test['item_id'] = test['item_id'].astype(int)

In [19]:
# т.е. сразу знаем количество и в каких пределах изменяется user_id и video_id
num_videos = train['item_id'].nunique()
num_users = train['user_id'].nunique()

print('Количество уникальных movie_id', num_videos)
print('Количество уникальных user_id', num_users)

Количество уникальных movie_id 25456
Количество уникальных user_id 18892


In [20]:
def prepare_hetero_data(df) -> HeteroData:
    data = HeteroData()
    users = df['user_id'].unique()
    num_users = len(users)
    num_items = df['item_id'].max() + 1
    feedback_types = df['target'].unique().tolist()

    # узлы
    data['user'].node_id = torch.arange(num_users)
    data['item'].node_id = torch.arange(num_items)
    for ft in feedback_types:
        data[ft].node_id = torch.arange(num_users)

    # ребра item<->feedback->user
    for ft in feedback_types:
        mask = df['target'] == ft
        items = torch.LongTensor(df.loc[mask, 'item_id'].values)
        users_idx = torch.LongTensor(df.loc[mask, 'user_id'].values)
        # item -> fb
        data['item', f'to_feedback_{ft}', ft].edge_index = torch.stack([items, users_idx], dim=0)
        # fb -> item
        data[ft, f'feedback_to_item_{ft}', 'item'].edge_index = torch.stack([users_idx, items], dim=0)
        # fb -> user (1:1)
        idx = torch.arange(num_users)
        data[ft, f'to_user_{ft}', 'user'].edge_index = torch.stack([idx, idx], dim=0)

    # user <-> user
    # data['user', 'interacts', 'user'].edge_index = build_full_graph(num_users)
    return data

In [21]:
class HeteroGNN(nn.Module):
    def __init__(self,
                 num_users: int,
                 num_items: int,
                 feedback_types: list,
                 emb_dim: int = 32,
                 hidden_dim: int = 16,
                 heads: int = 2,
                 dropout: float = 0.2):
        super().__init__()
        self.feedback_types = feedback_types
        self.pos_types = ['implicit_positive', 'explicit_positive']
        self.neg_types = ['implicit_negative', 'expliсit_negative']

        self.user_emb = nn.Embedding(num_users, emb_dim)
        self.item_emb = nn.Embedding(num_items, emb_dim)
        self.fb_emb   = nn.ModuleDict({
            ft: nn.Embedding(num_users, emb_dim) for ft in feedback_types
        })

        conv1, conv2 = {}, {}
        for ft in feedback_types:
            # ft -> user
            conv1[(ft, f'to_user_{ft}', 'user')] = GATConv(emb_dim, hidden_dim,
                                                      heads=heads,
                                                      add_self_loops=False)
            conv2[(ft, f'to_user_{ft}', 'user')] = GATConv(hidden_dim*heads, emb_dim,
                                                      heads=1,
                                                      add_self_loops=False)
            # ft -> item
            conv1[(ft, f'feedback_to_item_{ft}', 'item')] = GATConv(emb_dim, hidden_dim,
                                                       heads=heads,
                                                       add_self_loops=False)
            conv2[(ft, f'feedback_to_item_{ft}', 'item')] = GATConv(hidden_dim*heads, emb_dim,
                                                       heads=1,
                                                       add_self_loops=False)

            # item -> ft
            conv1[('item', f'to_feedback_{ft}', ft)] = GATConv(emb_dim, hidden_dim,
                                                       heads=heads,
                                                       add_self_loops=False)
            conv2[('item', f'to_feedback_{ft}', ft)] = GATConv(hidden_dim*heads, emb_dim,
                                                       heads=1,
                                                       add_self_loops=False)

        self.conv1 = HeteroConv(conv1, aggr='mean')
        self.conv2 = HeteroConv(conv2, aggr='mean')

        # LayerNorm и Dropout
        types = ['user', 'item'] + feedback_types
        self.norm1 = nn.ModuleDict({t: nn.LayerNorm(hidden_dim*heads) for t in types})
        self.norm2 = nn.ModuleDict({t: nn.LayerNorm(emb_dim) for t in types})
        self.dropout = nn.Dropout(dropout)

    def forward(self, data):
        x = {
            'user': self.user_emb(data['user'].node_id),
            'item': self.item_emb(data['item'].node_id)
        }
        for ft in self.feedback_types:
            x[ft] = self.fb_emb[ft](data[ft].node_id)

        h1 = self.conv1(x, data.edge_index_dict)
        for t, h in h1.items():
            h1[t] = self.dropout(F.leaky_relu(self.norm1[t](h)))

        h2 = self.conv2(h1, data.edge_index_dict)
        out = {}
        for t, h in h2.items():
            out[t] = self.norm2[t](h)
            
        return out['user']

In [22]:
data = prepare_hetero_data(train)
data

HeteroData(
  user={ node_id=[18892] },
  item={ node_id=[25456] },
  implicit_negative={ node_id=[18892] },
  explicit_positive={ node_id=[18892] },
  implicit_positive={ node_id=[18892] },
  expliсit_negative={ node_id=[18892] },
  (item, to_feedback_implicit_negative, implicit_negative)={ edge_index=[2, 258651] },
  (implicit_negative, feedback_to_item_implicit_negative, item)={ edge_index=[2, 258651] },
  (implicit_negative, to_user_implicit_negative, user)={ edge_index=[2, 18892] },
  (item, to_feedback_explicit_positive, explicit_positive)={ edge_index=[2, 359463] },
  (explicit_positive, feedback_to_item_explicit_positive, item)={ edge_index=[2, 359463] },
  (explicit_positive, to_user_explicit_positive, user)={ edge_index=[2, 18892] },
  (item, to_feedback_implicit_positive, implicit_positive)={ edge_index=[2, 430342] },
  (implicit_positive, feedback_to_item_implicit_positive, item)={ edge_index=[2, 430342] },
  (implicit_positive, to_user_implicit_positive, user)={ edge_index

In [23]:
train.item_id.nunique(), train.item_id.min(), train.item_id.max()

(25456, 0, 25455)

In [24]:
num_users = len(train['user_id'].unique())
num_items = train['item_id'].max() + 1
feedback_types = train['target'].unique().tolist()
model = HeteroGNN(num_users, num_items, feedback_types)

In [25]:
model

HeteroGNN(
  (user_emb): Embedding(18892, 32)
  (item_emb): Embedding(25456, 32)
  (fb_emb): ModuleDict(
    (implicit_negative): Embedding(18892, 32)
    (explicit_positive): Embedding(18892, 32)
    (implicit_positive): Embedding(18892, 32)
    (expliсit_negative): Embedding(18892, 32)
  )
  (conv1): HeteroConv(num_relations=12)
  (conv2): HeteroConv(num_relations=12)
  (norm1): ModuleDict(
    (user): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
    (item): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
    (implicit_negative): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
    (explicit_positive): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
    (implicit_positive): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
    (expliсit_negative): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
  )
  (norm2): ModuleDict(
    (user): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
    (item): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
    (

In [26]:
model(data)

tensor([[-1.5367, -0.9751,  0.0813,  ...,  0.7928, -1.2291,  0.9374],
        [ 0.7417, -1.3958,  1.3326,  ...,  0.7361,  0.5109, -0.2695],
        [ 0.5076, -0.3100, -0.0729,  ...,  1.4869,  0.7861, -1.5773],
        ...,
        [ 0.7925,  0.3648,  1.4548,  ...,  1.8573,  0.3579,  0.3480],
        [ 0.4220, -1.3121,  1.1391,  ...,  0.1881, -0.0039, -0.5980],
        [ 0.0363, -1.6866,  2.1876,  ...,  0.0029, -0.2593,  0.4358]],
       grad_fn=<NativeLayerNormBackward0>)

In [27]:
test_df = test[['user_id', 'item_id']]
interactions = test_df.rename(columns={
    'user_id': Columns.User,
    'item_id': Columns.Item,
})

viewed_items = train.groupby("user_id")["item_id"].agg(set).to_dict()

In [37]:
def train_model(model: HeteroGNN,
                train_data: HeteroData,
                edge_type: tuple,
                num_epochs: int = 10,
                lr: float = 1e-3,
                batch_size: int = 1024,
                device: str = None,
                print_every: int = 100,
                test_every: int = 500,
                top_k: int = 10,
                test_batch_size=2048) -> HeteroGNN:
    device = device or ('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    train_data = train_data.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    src, dst = train_data[edge_type].edge_index
    num_train = src.size(0)
    test_top_k = top_k * 300
    total_steps = 4370
    print(f"Num of training examples: {num_train}")
    for epoch in range(401, 501):
        model.train()
        perm = torch.randperm(num_train, device=device)
        total_loss = 0.0
        step = 0
        for i in range(0, num_train, batch_size):
            idx = perm[i:i + batch_size]
            users = dst[idx]
            pos_items = src[idx]
            neg_items = torch.randint(0, model.item_emb.num_embeddings,
                                      size=pos_items.size(), device=device)
            optimizer.zero_grad()
            user_embs = model(train_data)[users]
            pos_emb = model.item_emb(pos_items)
            neg_emb = model.item_emb(neg_items)
            pos_score = (user_embs * pos_emb).sum(dim=1)
            neg_score = (user_embs * neg_emb).sum(dim=1)
            loss = -torch.log(torch.sigmoid(pos_score - neg_score) + 1e-15).mean()
            loss.backward()
            optimizer.step()
            total_loss += loss.item() * users.size(0)
            step += 1

            experiment.log_metric('Train BPR Loss vs step', loss.item(), step=total_steps)
            if step % print_every == 0 or step == 1:
                avg_loss = total_loss /  (step * batch_size)
                current_lr = optimizer.param_groups[0]['lr']
                d = (pos_score - neg_score).detach().cpu()
                print(f"Epoch {epoch}, Step {step}, LR: {current_lr:.6f}, Current Loss: {loss.item():.4f}, Avg Loss: {avg_loss:.4f}")
                print(f"Diff stats — min: {d.min():.4f}, max: {d.max():.4f}, mean: {d.mean():.4f}, std: {d.std():.4f}")
                print()

                experiment.log_metric('Diff stats (mean) vs step', d.mean(), step=total_steps)
                experiment.log_metric('Diff stats (std) vs step', d.std(), step=total_steps)

            del user_embs, pos_emb, neg_emb, pos_score, neg_score
            gc.collect()
            torch.cuda.empty_cache()
            
            if step % test_every == 0 or step == 1:
                model.eval()
                # model.to('cpu')
                with torch.no_grad():
                    user_emb = model(train_data)
                    item_emb = model.item_emb.weight
                    item_emb_t = item_emb.t().detach()
                    del item_emb
                    gc.collect()

                    scores = []
                    for i in range(0, user_emb.shape[0], test_batch_size):
                        user_e = user_emb[i:i+test_batch_size]
                        rating = torch.mm(user_e, item_emb_t)
                        _, topk_items = torch.topk(rating, k=test_top_k, dim=1)
                        scores.extend(topk_items)

                        del user_e, rating
                        gc.collect()

                    scores = torch.cat(scores, dim=0)
                    scores = scores.reshape((num_users, test_top_k))
                    topk_np = scores.cpu().numpy()

                    filtered_topk = []
                    for u_idx in range(num_users):
                        seen = viewed_items[u_idx]
                        recs = topk_np[u_idx, :]
                        mask = np.isin(recs, list(seen), invert=True)
                        filtered = recs[mask][:top_k] 
                        filtered_topk.append(filtered)
                    
                    users = []
                    items = []
                    ranks = []
                    for u_idx in range(num_users):
                        recs = filtered_topk[u_idx]
                        for rank, item in enumerate(recs[:top_k], 1):
                            users.append(u_idx)
                            items.append(item)
                            ranks.append(rank)
                    
                    reco_df = pd.DataFrame({
                        Columns.User: users,
                        Columns.Item: items,
                        Columns.Rank: ranks,
                    })

                    metrics = {
                        'map@10': MAP(k=10),
                        'precision@10': Precision(k=10),
                        'recall@10': Recall(k=10),
                        'ndcg@10': NDCG(k=10)
                    }
                    
                    results = calc_metrics(
                        metrics=metrics,
                        reco=reco_df,
                        interactions=interactions,
                    )
                print(f"Step/epoch {step}/{epoch}, Test metrics:")
                for key, value in results.items():
                    print(f"{key}: {value:.9f}")
                    experiment.log_metric(f"Test {key} vs step", value, step=total_steps)
                del scores, topk_np
                gc.collect()

                model.to(device)
                model.train()
                train_data.to(device)
            total_steps += 1
        epoch_loss = total_loss / num_train
        experiment.log_metric(f'Train Loss vs epoch', epoch_loss, epoch=epoch - 1)
        print(f"Epoch {epoch} completed, Train Loss: {epoch_loss:.4f}")
    return model

In [29]:
experiment.log_parameters(hyperparameters)

In [30]:
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

In [31]:
edge_type = hyperparameters['train_edge_type']
num_epochs = hyperparameters['train_num_epochs']
lr = hyperparameters['train_lr']
batch_size = hyperparameters['train_batch_size']
print_every = hyperparameters['train_print_every']
test_every = hyperparameters['train_test_every']
top_k = hyperparameters['test_topk']
test_batch_size = hyperparameters['test_batch_size']
model = train_model(model,
                    data,
                    edge_type=edge_type,
                    num_epochs=num_epochs,
                    lr=lr,
                    batch_size=batch_size,
                    print_every=print_every,
                    test_every=test_every,
                    top_k=top_k,
                    test_batch_size=test_batch_size)

Num of training examples: 359463
Epoch 1, Step 1, LR: 0.000300, Current Loss: 3.2475, Avg Loss: 3.2475
Diff stats — min: -33.7539, max: 32.2192, mean: 0.0062, std: 7.9287

Step/epoch 1/1, Test metrics:
precision@10: 0.000290698
recall@10: 0.000292349
ndcg@10: 0.000313494
map@10: 0.000098763
Epoch 1 completed, Train Loss: 3.2435
Epoch 2, Step 1, LR: 0.000300, Current Loss: 3.2174, Avg Loss: 3.2174
Diff stats — min: -33.8460, max: 39.4691, mean: 0.0192, std: 7.8902

Step/epoch 1/2, Test metrics:
precision@10: 0.000330338
recall@10: 0.000331990
ndcg@10: 0.000320361
map@10: 0.000091778
Epoch 2 completed, Train Loss: 3.1970
Epoch 3, Step 1, LR: 0.000300, Current Loss: 3.1812, Avg Loss: 3.1812
Diff stats — min: -30.3655, max: 36.4971, mean: 0.0443, std: 7.8198

Step/epoch 1/3, Test metrics:
precision@10: 0.000343552
recall@10: 0.000345203
ndcg@10: 0.000327438
map@10: 0.000092010
Epoch 3 completed, Train Loss: 3.1291
Epoch 4, Step 1, LR: 0.000300, Current Loss: 3.0913, Avg Loss: 3.0913
Diff s

In [34]:
model = train_model(model,
                    data,
                    edge_type=edge_type,
                    num_epochs=num_epochs,
                    lr=lr,
                    batch_size=batch_size,
                    print_every=print_every,
                    test_every=test_every,
                    top_k=top_k,
                    test_batch_size=test_batch_size)

Num of training examples: 359463
Epoch 101, Step 1, LR: 0.000300, Current Loss: 0.6207, Avg Loss: 0.6207
Diff stats — min: -11.7706, max: 20.3186, mean: 2.1129, std: 3.1475

Step/epoch 1/101, Test metrics:
precision@10: 0.000647463
recall@10: 0.000648931
ndcg@10: 0.000620489
map@10: 0.000172451
Epoch 101 completed, Train Loss: 0.6086
Epoch 102, Step 1, LR: 0.000300, Current Loss: 0.6040, Avg Loss: 0.6040
Diff stats — min: -14.0433, max: 18.4533, mean: 2.1102, std: 3.0854

Step/epoch 1/102, Test metrics:
precision@10: 0.000667283
recall@10: 0.000668751
ndcg@10: 0.000638691
map@10: 0.000177165
Epoch 102 completed, Train Loss: 0.5977
Epoch 103, Step 1, LR: 0.000300, Current Loss: 0.6056, Avg Loss: 0.6056
Diff stats — min: -10.7141, max: 18.7762, mean: 2.0826, std: 3.0604

Step/epoch 1/103, Test metrics:
precision@10: 0.000693710
recall@10: 0.000695179
ndcg@10: 0.000660836
map@10: 0.000182889
Epoch 103 completed, Train Loss: 0.5868
Epoch 104, Step 1, LR: 0.000300, Current Loss: 0.5783, Avg

In [36]:
model = train_model(model,
                    data,
                    edge_type=edge_type,
                    num_epochs=num_epochs,
                    lr=lr,
                    batch_size=batch_size,
                    print_every=print_every,
                    test_every=test_every,
                    top_k=top_k,
                    test_batch_size=test_batch_size)

Num of training examples: 359463
Epoch 201, Step 1, LR: 0.000300, Current Loss: 0.3224, Avg Loss: 0.3224
Diff stats — min: -5.7151, max: 15.9051, mean: 2.3609, std: 2.2275

Step/epoch 1/201, Test metrics:
precision@10: 0.006309461
recall@10: 0.006310195
ndcg@10: 0.006329981
map@10: 0.001930477
Epoch 201 completed, Train Loss: 0.3147
Epoch 202, Step 1, LR: 0.000300, Current Loss: 0.3099, Avg Loss: 0.3099
Diff stats — min: -7.1298, max: 17.2872, mean: 2.4001, std: 2.2101

Step/epoch 1/202, Test metrics:
precision@10: 0.006362315
recall@10: 0.006363049
ndcg@10: 0.006383212
map@10: 0.001944052
Epoch 202 completed, Train Loss: 0.3134
Epoch 203, Step 1, LR: 0.000300, Current Loss: 0.3108, Avg Loss: 0.3108
Diff stats — min: -5.8505, max: 15.2460, mean: 2.4172, std: 2.2253

Step/epoch 1/203, Test metrics:
precision@10: 0.006461416
recall@10: 0.006462151
ndcg@10: 0.006448063
map@10: 0.001963248
Epoch 203 completed, Train Loss: 0.3130
Epoch 204, Step 1, LR: 0.000300, Current Loss: 0.3106, Avg Lo

In [38]:
model = train_model(model,
                    data,
                    edge_type=edge_type,
                    num_epochs=num_epochs,
                    lr=lr,
                    batch_size=batch_size,
                    print_every=print_every,
                    test_every=test_every,
                    top_k=top_k,
                    test_batch_size=test_batch_size)

Num of training examples: 359463
Epoch 401, Step 1, LR: 0.000300, Current Loss: 0.2649, Avg Loss: 0.2649
Diff stats — min: -6.2777, max: 18.6241, mean: 3.0737, std: 2.6345

Step/epoch 1/401, Test metrics:
precision@10: 0.006923890
recall@10: 0.006926092
ndcg@10: 0.007116629
map@10: 0.002229065
Epoch 401 completed, Train Loss: 0.2647
Epoch 402, Step 1, LR: 0.000300, Current Loss: 0.2583, Avg Loss: 0.2583
Diff stats — min: -6.3039, max: 17.7222, mean: 3.0789, std: 2.6084

Step/epoch 1/402, Test metrics:
precision@10: 0.007003171
recall@10: 0.007004639
ndcg@10: 0.007160075
map@10: 0.002229840
Epoch 402 completed, Train Loss: 0.2636
Epoch 403, Step 1, LR: 0.000300, Current Loss: 0.2621, Avg Loss: 0.2621
Diff stats — min: -6.2640, max: 17.7641, mean: 3.0808, std: 2.6267

Step/epoch 1/403, Test metrics:
precision@10: 0.006917283
recall@10: 0.006919486
ndcg@10: 0.007109042
map@10: 0.002224479
Epoch 403 completed, Train Loss: 0.2633
Epoch 404, Step 1, LR: 0.000300, Current Loss: 0.2637, Avg Lo

KeyboardInterrupt: 

In [39]:
torch.save(model, "gnn_model_mvl.model")
from IPython.display import FileLink

FileLink('gnn_model_mvl.model')

/kaggle/working/gnn_model_mvl.model

In [40]:
# del model
gc.collect()
torch.cuda.empty_cache()

In [41]:
log_model(
    experiment=experiment,
    model=model,
    model_name="GNN",
)

In [42]:
experiment.end()

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : emogat-books
COMET INFO:     url                   : https://www.comet.com/annanet/gnn-recommender/bed3511a1e964e0594d0fb04c7d195b3
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     Diff stats (mean) vs step [448] : (0.006162459030747414, 3.168837785720825)
COMET INFO:     Diff stats (std) vs step [448]  : (2.0434560775756836, 7.928686618804932)
COMET INFO:     Test map@10 vs step [448]       : (8.660892563508843e-05, 0.002307110417575534)
COMET INFO:     Test ndcg@10 vs step [448]      : (0.0003134935326079566, 0.0073784688986630945)
COMET INFO:     Test precision@10 vs step [448] : (0.0002906976744186047, 0.007280655391120508)
COMET INFO:  